In [15]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import scipy

from sklearn.metrics import accuracy_score

import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
data  = torch.load('Datasets/NASA_turbofan_get_engine/data.pt')
# Flatten the tensor to size 236x2400
flattened_tensor = data.view(236, -1)

# Convert to pandas DataFrame directly from the tensor
num_timesteps = data.shape[1]
num_features_per_timestep = data.shape[2]
df = pd.DataFrame(flattened_tensor.tolist(), columns=[f'time_{t}_feature_{f}' for t in range(num_timesteps) for f in range(num_features_per_timestep)])

print(df.shape) 

label = torch.load('Datasets/NASA_turbofan_get_engine/label.pt')
df['target'] = label.tolist()

(236, 2400)


choosing most correlated features

In [9]:
cor = df.corr()['target'].sort_values(ascending=False).drop(['target'])
features = cor[cor>0.1]

In [14]:
type(features.index)

pandas.core.indexes.base.Index

In [16]:
# Train-test split
target = df['target']
features_df = df[features.index]
X_train, X_test, y_train, y_test = train_test_split(features_df.values, target, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train TabNet model
model = TabNetClassifier()
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric=['accuracy'])

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


RuntimeError: Could not infer dtype of numpy.float32